In [2]:

from scipy.stats import norm

In [35]:
def black_scholes_put(current_price, strike, time_to_expiry, risk, sigma_volatility, contracts):
    """Calculates the price of a put option using the Black-Scholes model.

    Args:
        current_price: Current stock price
        strike: Strike price
        time_to_expiry: Time to expiration (in years)
        risk: Risk-free interest rate
        sigma_volatility: Volatility

    Returns:
        The price of the put option.
    """

    d1 = (np.log(current_price / strike) + (risk + 0.5 * sigma_volatility ** 2) * time_to_expiry) / (
            sigma_volatility * np.sqrt(time_to_expiry))
    d2 = d1 - sigma_volatility * np.sqrt(time_to_expiry)

    put_price = strike * np.exp(-risk * sigma_volatility) * norm.cdf(-d2) - current_price * norm.cdf(-d1)
    delta = -norm.cdf(-d1)  # Negative sign for put options
    profit = max(0, (strike - current_price) * contracts)  # Profit if exercised at current price
    return put_price, delta, profit


In [36]:
"""
Binomial Tree Model
"""

import numpy as np


def binomial_tree_american_put(S, K, T, r, sigma, N, contracts):
    """Calculates the price of an American put option using a binomial tree.

    Args:
        S: Current stock price
        K: Strike price
        T: Time to expiration (in years)
        r: Risk-free interest rate
        sigma: Volatility
        N: Number of time steps in the binomial tree

    Returns:
        The price of the American put option.
    """
    N = 50  # Adjust for desired accuracy
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Create stock price tree
    stock_tree = np.zeros((N + 1, N + 1))
    stock_tree[0, 0] = S
    for i in range(1, N + 1):
        stock_tree[i, 0] = stock_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_tree[i, j] = stock_tree[i - 1, j - 1] * d

    # Calculate option values at expiration
    option_tree = np.maximum(K - stock_tree, 0)

    # Work backward through the tree, considering early exercise
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_tree[i, j] = max(
                np.exp(-r * dt) * (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j + 1]),
                K - stock_tree[i, j]
            )

    delta = (option_tree[1, 1] - option_tree[1, 0]) / (stock_tree[1, 1] - stock_tree[1, 0])

    profit = max(0, (strike - current_price) * contracts)  # Profit if exercised at current price

    return option_tree[0, 0], delta, profit


In [66]:

# Example usage - calculating the price before it reaches my strike price
current_price = 128  # Current stock price at the time of exercise
strike = 98 # Strike price
time_to_expiry = 365  # 30/365 Remaining time to expiration (if applicable)
risk = 0.008  # Risk-free interest rate
sigma_volatility = 0.5
contracts = 100
premium_paid = 2.66  # Premium paid per contract
N = 50  # Adjust for desired accuracy
# put option price: 91.5405383687267
# Black-Scholes Put Option Price: 91.5405383687267
# Binomial Tree American Put Option Price: 65.0750230971521

In [68]:
put_price, delta_bs, profit = black_scholes_put(current_price, strike, time_to_expiry, risk, sigma_volatility,
                                                contracts)
print("Put option price:", put_price)

print("Black-Scholes Put Option Price:", put_price)
# Profit = (Strike Price - Current Stock Price) * Number of Contracts - Premium Paid
put_price, delta_bt, profit = binomial_tree_american_put(current_price, strike, time_to_expiry, risk, sigma_volatility,
                                                         N, contracts)
print("Binomial Tree American Put Option Price:", put_price)

Put option price: 97.60832857205732
Black-Scholes Put Option Price: 97.60832857205732
Binomial Tree American Put Option Price: 73.31141496830648


In [70]:
f'{97 * 0.91:.4f}'
f'{5.39 * 1.18:.4f}'

'6.3602'

In [77]:
# Example usage
S = 70  # Current stock price at the time of exercise
K = 88  # Strike price
T = 365 / 365  # Remaining time to expiration (if applicable)
r = 0.008
sigma = 0.5
contracts = 58
premium_paid = 6.36  # Premium paid per contract
N = 50  # Adjust for desired accuracy
# Black-Scholes
_, _, profit_bs = black_scholes_put(S, K, T, r, sigma, contracts)
# premium_paid = premium_paid * contracts  # Total premium paid 
profit_bs_minus_paid = profit_bs - premium_paid * contracts  # Subtract the premium paid
print("premium_paid:", premium_paid * contracts +20)
print("profit - premium_paid:", profit_bs_minus_paid)

print("Black-Scholes Profit if Exercised Now:", profit_bs)

# Binomial Tree

_, _, profit_bt = binomial_tree_american_put(S, K, T, r, sigma, N, contracts)
profit_bt -= premium_paid * contracts  # Subtract the premium paid
print("Binomial Tree Profit if Exercised Now:", profit_bt)

premium_paid: 388.88
profit - premium_paid: 675.12
Black-Scholes Profit if Exercised Now: 1044
Binomial Tree Profit if Exercised Now: -368.88
